<a href="https://colab.research.google.com/github/nicolai5965/Predictive-Text-Generation-with-GRU-Model/blob/main/Predictive_Text_Generation_with_GRU_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from nltk.corpus import words

# Download the words corpus if you haven't done so already
import nltk
nltk.download('words')

import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from google.colab import drive
# Mount Google Drive to load the dataset
drive.mount('/content/drive') 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# Define the file path and load the train and test data
filepath = '/content/drive/My Drive/Colab Notebooks/Machine Learning/TensorFlow/GRU/'

In [ ]:
df = pd.read_csv(f'{filepath}fake_or_real_news.csv')


In [ ]:
display(df)

In [ ]:
def prepare_text_data(df, column_name, data_proportion, test_size, fixed_sequence_len, fixed_n_gram_len):
    # Extract text data
    text = list(df[column_name].values)

    # Determine the number of samples to use
    num_samples = int(len(text) * data_proportion)

    # Use only the specified proportion of samples
    text = text[:num_samples]

    # Lowercasing: Convert all text to lowercase.
    text = [t.lower() for t in text]

    # Split data into training and validation sets
    train_text, val_text = train_test_split(text, test_size=test_size, random_state=42)

    # Define tokenizer
    tokenizer = Tokenizer()

    # Fit tokenizer on training text data
    tokenizer.fit_on_texts(train_text)

    # Total number of words before removal
    total_words_before_removal = len(tokenizer.word_index) + 1

    # Remove words that are not recognized as English words
    english_words = set(words.words())
    tokenizer.word_index = {word: index for word, index in tokenizer.word_index.items() if word in english_words}
    removed_words = list(set(tokenizer.word_index.keys()) - english_words)

    # Update total_words after removal
    total_words = len(tokenizer.word_index) + 1

    # Function to convert text data to sequence of tokens
    def text_to_sequences(text_data):
        input_sequences = []
        for line in text_data:
            token_list = tokenizer.texts_to_sequences([line])[0]
            for i in range(0, len(token_list) - fixed_n_gram_len + 1):
                n_gram_sequence = token_list[i:i + fixed_n_gram_len]
                input_sequences.append(n_gram_sequence)
        return input_sequences

    # Convert training and validation text data to sequences
    train_sequences = text_to_sequences(train_text)
    val_sequences = text_to_sequences(val_text)

    # Pad sequences
    train_sequences = np.array(pad_sequences(train_sequences, maxlen=fixed_sequence_len, padding='pre'))
    val_sequences = np.array(pad_sequences(val_sequences, maxlen=fixed_sequence_len, padding='pre'))

    # Create predictors and labels for both training and validation sets
    train_x, train_y = train_sequences[:, :-1], train_sequences[:, -1]
    val_x, val_y = val_sequences[:, :-1], val_sequences[:, -1]

    # Convert to numpy array and set out-of-range indices to 0
    train_y, val_y = np.array(train_y), np.array(val_y)
    train_y[train_y >= total_words], val_y[val_y >= total_words] = 0, 0

    # Convert labels to categorical
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=total_words)
    val_y = tf.keras.utils.to_categorical(val_y, num_classes=total_words)

    return (train_x, train_y), (val_x, val_y), total_words, tokenizer, removed_words

In [ ]:
# Custom GRU cell
class GRUCell(Layer):
    def __init__(self, units, activation='tanh', kernel_regularizer=None, recurrent_regularizer=None, **kwargs):
        super(GRUCell, self).__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)

    def build(self, input_shape):
        self.kernel_z = self.add_weight(shape=(input_shape[-1], self.units),
                                        initializer='glorot_uniform',
                                        name='kernel_z',
                                        regularizer=self.kernel_regularizer)
        self.recurrent_kernel_z = self.add_weight(shape=(self.units, self.units),
                                                  initializer='orthogonal',
                                                  name='recurrent_kernel_z',
                                                  regularizer=self.recurrent_regularizer)
        self.bias_z = self.add_weight(shape=(self.units,), initializer='zeros', name='bias_z')

        self.kernel_r = self.add_weight(shape=(input_shape[-1], self.units),
                                        initializer='glorot_uniform',
                                        name='kernel_r',
                                        regularizer=self.kernel_regularizer)
        self.recurrent_kernel_r = self.add_weight(shape=(self.units, self.units),
                                                  initializer='orthogonal',
                                                  name='recurrent_kernel_r',
                                                  regularizer=self.recurrent_regularizer)
        self.bias_r = self.add_weight(shape=(self.units,), initializer='zeros', name='bias_r')

        self.kernel_h = self.add_weight(shape=(input_shape[-1], self.units),
                                        initializer='glorot_uniform',
                                        name='kernel_h',
                                        regularizer=self.kernel_regularizer)
        self.recurrent_kernel_h = self.add_weight(shape=(self.units, self.units),
                                                  initializer='orthogonal',
                                                  name='recurrent_kernel_h',
                                                  regularizer=self.recurrent_regularizer)
        self.bias_h = self.add_weight(shape=(self.units,), initializer='zeros', name='bias_h')

        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]

        # Update gate calculation
        z = self.update_gate(inputs, prev_output)

        # Reset gate calculation
        r = self.reset_gate(inputs, prev_output)

        # Hidden state candidate
        h_candidate = self.hidden_state_candidate(inputs, prev_output, r)

        # Final output calculation
        output = self.final_output(prev_output, h_candidate, z)

        return output, [output]

    def update_gate(self, inputs, prev_output):
        return tf.keras.activations.sigmoid(tf.matmul(inputs, self.kernel_z) + tf.matmul(prev_output, self.recurrent_kernel_z) + self.bias_z)

    def reset_gate(self, inputs, prev_output):
        return tf.keras.activations.sigmoid(tf.matmul(inputs, self.kernel_r) + tf.matmul(prev_output, self.recurrent_kernel_r) + self.bias_r)

    def hidden_state_candidate(self, inputs, prev_output, r):
        return self.activation(tf.matmul(inputs, self.kernel_h) + tf.matmul(r * prev_output, self.recurrent_kernel_h) + self.bias_h)

    def final_output(self, prev_output, h_candidate, z):
        return (1 - z) * prev_output + z * h_candidate

    def get_initial_state(self, inputs):
        batch_size = tf.shape(inputs)[0]
        return [tf.zeros([batch_size, self.units], dtype=inputs.dtype)]


In [ ]:
# Custom GRU layer
class GRU(Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super(GRU, self).__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences

    def call(self, inputs):
        # Initialize the state
        initial_state = self.cell.get_initial_state(inputs)
        states = initial_state

        # Process the input sequence
        outputs = []
        for t in range(inputs.shape[1]):
            input_t = inputs[:, t]
            output, states = self.cell(input_t, states)
            outputs.append(output)

        # Stack the outputs
        outputs = tf.stack(outputs, axis=1)

        if self.return_sequences:
            return outputs
        else:
            return outputs[:, -1]

In [ ]:
class Model(tf.keras.Model):
    def __init__(self, embedding_dim, hidden_units, max_features, l2_reg):
        super(Model, self).__init__()
        self.embedding = Embedding(input_dim=max_features, output_dim=embedding_dim)
        self.gru_cell = GRUCell(units=hidden_units, kernel_regularizer=regularizers.l2(l2_reg), recurrent_regularizer=regularizers.l2(l2_reg))
        self.gru = GRU(self.gru_cell, return_sequences=False)
        self.dense = Dense(max_features, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.gru(x)
        return self.dense(x)

In [ ]:
# Data preprocessing configuration
preprocessing_config = {
    'data_proportion': 0.04,
    'text_split_size': 0.2,
    'fixed_sequence_len': 25,
    'fixed_n_gram_len': 20
}


In [ ]:
# Prepare the text data
(train_x, train_y), (val_x, val_y), total_words, tokenizer, removed_words = prepare_text_data(df, 
                                                                    'text', 
                                                                    preprocessing_config['data_proportion'], 
                                                                    preprocessing_config['text_split_size'],
                                                                    preprocessing_config['fixed_sequence_len'], 
                                                                    preprocessing_config['fixed_n_gram_len'])

In [ ]:
# Display shape of predictors and label
print("Train Predictors shape:", train_x.shape)
print("Train Label shape:", train_y.shape)
print("Validation Predictors shape:", val_x.shape)
print("Validation Label shape:", val_y.shape)

# Display an example predictor and label
print("Example Predictor:", train_x[0])
print("Example Label:", train_y[0])

# Display the total number of unique words
print("Total number of unique words:", total_words)

# Output removed words
if len(removed_words) > 0:
    print("Number of words removed:", len(removed_words))
    print("Removed words:", removed_words)
else:
    print("No words were removed.")

In [ ]:
# Display distribution of sequence lengths
sequence_lengths = [len(seq) for seq in np.concatenate([train_x, val_x])]
plt.hist(sequence_lengths, bins=30)
plt.title('Distribution of Sequence Lengths')
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.show()

# Get word counts
word_counts = tokenizer.word_counts
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Display most common and least common words
print("10 most common words:", sorted_word_counts[:10])
print("10 least common words:", sorted_word_counts[-10:])

# Count and display number of words that appear only once
single_appearance_words = len([count for word, count in word_counts.items() if count == 1])
print("Number of words that appear only once:", single_appearance_words)

# Display example of original text and its tokenized equivalent
example_text = df['text'][0]  # As the original text isn't directly returned by the function anymore
example_tokens = tokenizer.texts_to_sequences([example_text])[0]
print("Example original text:", example_text[:20])
print("Example tokenized text:", example_tokens[:20])


In [ ]:
# Model configuration
model_config = {
    'embedding_dim': 100,
    'hidden_units': 32,
    'max_features': total_words,
    'l2_reg': 1e-4,
    'optimizer': 'RMSprop',
    'learning_rate': 0.01,
    'loss': 'categorical_crossentropy',
    'metrics': ['accuracy'],
    'batch_size': 256,
    'epochs': 10,
    'validation_split': 0.2,
    'patience': 5,
    'verbose': 2
}

In [ ]:
# Create and compile the model
model = Model(model_config['embedding_dim'], model_config['hidden_units'], model_config['max_features'], model_config['l2_reg'])
model.compile(optimizer=RMSprop(learning_rate=model_config['learning_rate']), 
              loss=model_config['loss'], 
              metrics=model_config['metrics'])

# Define the early stopping callback with the desired parameters
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=model_config['patience'], 
                               restore_best_weights=True)

# Specify the path where you want to save the model
checkpoint_filepath = '/content/drive/My Drive/Colab Notebooks/Machine Learning/TensorFlow/GRU/GRU_best_model_callback/'

# Create a ModelCheckpoint callback that saves the weights only during training
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


In [ ]:
# Train the model
history = model.fit(train_x, 
                    train_y, 
                    epochs=model_config['epochs'], 
                    batch_size=model_config['batch_size'], 
                    validation_data=(val_x, val_y), 
                    callbacks=[early_stopping, model_checkpoint_callback], 
                    verbose=model_config['verbose'])

In [ ]:
display(model.summary())

In [ ]:
def predict_next_word(model, tokenizer, input_text, fixed_sequence_len, n_best=3):
    # Tokenize the input text
    input_tokens = tokenizer.texts_to_sequences([input_text])[0]
    # Pad the tokens
    input_tokens = pad_sequences([input_tokens], maxlen=fixed_sequence_len-1, padding='pre')
    # Predict the probabilities for the next word
    probabilities = model.predict(input_tokens, verbose=0)
    # Get the indices of the n_best most probable next words
    top_indices = np.argpartition(probabilities[0], -n_best)[-n_best:]
    # Get these indices sorted by probability
    sorted_indices = top_indices[np.argsort(-probabilities[0][top_indices])]
    # Reverse the word index
    reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
    # Filter valid indices
    valid_indices = [i for i in sorted_indices if i in reverse_word_index and i != 0]  # <-- Modified line
    # Look up the words corresponding to the valid indices
    next_words = [reverse_word_index[i] for i in valid_indices]  # <-- Modified line
    # Look up the probabilities corresponding to the valid indices
    likelihoods = probabilities[0][valid_indices]  # <-- Modified line
    # Create a DataFrame with the words and their likelihoods
    df = pd.DataFrame({
        'Word': next_words,
        'Likelihood': [f"{likelihood * 100:.2f}%" for likelihood in likelihoods]
    })
    return df

best_words = 3

possible = predict_next_word(model, tokenizer, "The economic outlook for the next year has been released and it suggests that the growth rate will", preprocessing_config['fixed_sequence_len'])
display(possible)
possible = predict_next_word(model, tokenizer, "In the world of artificial intelligence, recent advancements have led to a renewed interest in developing systems that can", preprocessing_config['fixed_sequence_len'])
display(possible)
possible = predict_next_word(model, tokenizer, "Climate change is having a dramatic effect on global weather patterns, and scientists are warning that if we don't", preprocessing_config['fixed_sequence_len'])
display(possible)

In [ ]:
def generate_text(model, tokenizer, input_text, fixed_sequence_len, n_words, creativity=3):
    word_sequence = input_text.split()
    for _ in range(n_words):
        sub_sequence = " ".join(word_sequence[-fixed_sequence_len:])
        try:
            predictions = predict_next_word(model, tokenizer, sub_sequence, fixed_sequence_len, n_best=creativity)
            predictions.sort_values(by='Likelihood', ascending=False, inplace=True)
            top_predictions = predictions.head(creativity)['Word'].tolist()
            choice = random.choice(top_predictions)
        except Exception as e:
            print(f"Error during prediction: {e}")
            choice = random.choice(word_sequence)
        word_sequence.append(choice)
    return " ".join(word_sequence)


input_string = "The economic outlook for the next year has been released and it suggests that the growth rate will"
generated_text = generate_text(model, tokenizer, input_string, preprocessing_config['fixed_sequence_len'], n_words=100, creativity=3)
print(generated_text)
